# JAX primitives

*necula@google.com*, September 2019.

JAX implements certain transformations of Python functions, e.g., `jit`, `grad`,
`vmap`, or `pmap`. The Python functions to be transformed must be JAX-traceable, 
which means that as the Python function executes
the only operations it applies to the data are either inspections of data
attributes such as shape or type, or special operations called JAX primitives.
In particular, a JAX-traceable function is sometimes invoked by JAX with
abstract arguments. An example of a JAX abstract value is `ShapedArray(float32[2,2])`, 
which captures the type and the shape of values, but not the concrete data values.
JAX primitives know how to operate on both concrete data
values and on the JAX abstract values.


The JAX-transformed functions must themselves be JAX-traceable functions,
to ensure that these transformations
can be composed, e.g., `jit(jacfwd(grad(f)))`.

There are pre-defined JAX primitives corresponding to most XLA operations, 
e.g., add, matmul, sin, cos, indexing.
JAX comes with an implementation of numpy functions in terms of JAX primitives, which means that Python programs
using JAX’s implementation of numpy are JAX-traceable and therefore transformable.
Other libraries can be made JAX-traceable by implementing them in terms of JAX primitives.

The set of JAX primitives is extensible. Instead of reimplementing a function in terms of pre-defined JAX primitives,
one can define a new primitive that encapsulates the behavior of the function.

**The goal of this document is to explain the interface that a JAX primitive must support in order to allow JAX to perform all its transformations.**

Consider that we want to add to JAX support for a multiply-add function with three arguments, defined mathematically
as "ma(x, y, z) = x * y + z". 
This function operates on 3 identically-shaped tensors of floating point 
values and performs the opertions pointwise.




## Defining new functionality using existing primitives

The easiest way to define new functions is to write them in terms of JAX primitives, or in terms of other
functions that are themselves written using JAX primitives, e.g., those 
defined in the `jax.lax` module:

In [0]:
from jax import lax
from jax import api

def ma_lax(x, y, z):
  """Implementation of multiply-add using the jax.lax primitives."""
  return lax.add(lax.mul(x, y), z)


def sq_add_lax(a, b):
  """A square-add function using the newly defined multiply-add."""
  return ma_lax(a, a, b)

print("sq_add_lax = ", sq_add_lax(2., 10.))
# Differentiate w.r.t. the first argument
print("grad(sq_add_lax) = ", api.grad(sq_add_lax, argnums=0)(2.0, 10.))

sq_add_lax =  14.0
grad(sq_add_lax) =  4.0


In order to understand how JAX is using internally the primitives,
we add some helpers for tracing function calls.

In [0]:
#@title Helper functions (execute this cell)
import functools
import traceback

_indentation = 0
def _trace(msg=None):
    """Print a message at current indentation."""
    if msg is not None:
        print("  " * _indentation + msg)

def _trace_indent(msg=None):
    """Print a message and then indent the rest."""
    global _indentation
    _trace(msg)
    _indentation = 1 + _indentation

def _trace_unindent(msg=None):
    """Unindent then print a message."""
    global _indentation
    _indentation = _indentation - 1
    _trace(msg)

def trace(name):
  """A decorator for functions to trace arguments and results."""

  def trace_func(func):  # pylint: disable=missing-docstring
    def pp(v):
        """Print certain values more succinctly"""
        vtype = str(type(v))
        if "jax.lib.xla_bridge._JaxComputationBuilder" in vtype:
            return "<JaxComputationBuilder>"
        elif "jaxlib.xla_extension.XlaOp" in vtype:
            return "<XlaOp at 0x{:x}>".format(id(v))
        elif ("partial_eval.JaxprTracer" in vtype or
              "batching.BatchTracer" in vtype or
              "ad.JVPTracer" in vtype):
            return "Traced<{}>".format(v.aval)
        elif isinstance(v, tuple):
            return "({})".format(pp_values(v))
        else:
            return str(v)
    def pp_values(args):
        return ", ".join([pp(arg) for arg in args])
    
    @functools.wraps(func)
    def func_wrapper(*args):
      _trace_indent("call {}({})".format(name, pp_values(args)))
      res = func(*args)
      _trace_unindent("|<- {} = {}".format(name, pp(res)))
      return res

    return func_wrapper

  return trace_func

class expectNotImplementedError(object):
  """Context manager to check for NotImplementedError."""
  def __enter__(self): pass
  def __exit__(self, type, value, tb):
    global _indentation
    _indentation = 0
    if type is NotImplementedError:
      print("\nFound expected exception:")
      traceback.print_exc(limit=3)
      return True
    elif type is None:  # No exception
      assert False, "Expected NotImplementedError"
    else:
      return False

Instead of using `jax.lax` primitives directly, we can use other functions 
that are already written in terms of those primitives, such as those in `jax.numpy`: 

In [0]:
import jax.numpy as jnp
import numpy as onp

@trace("ma_numpy")
def ma_numpy(x, y, z):
    return jnp.add(jnp.multiply(x, y), z)

@trace("sq_add_numpy")
def sq_add_numpy(a, b):
    return ma_numpy(a, a, b)

print("\nNormal evaluation:")  
print("sq_add_numpy = ", sq_add_numpy(2., 10.))
print("\nGradient evaluation:")
print("grad(sq_add_numpy) = ", api.grad(sq_add_numpy)(2.0, 10.))


Normal evaluation:
call sq_add_numpy(2.0, 10.0)
  call ma_numpy(2.0, 2.0, 10.0)
  |<- ma_numpy = 14.0
|<- sq_add_numpy = 14.0
sq_add_numpy =  14.0

Gradient evaluation:
call sq_add_numpy(Traced<ConcreteArray(2.0)>, 10.0)
  call ma_numpy(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, 10.0)
  |<- ma_numpy = Traced<ConcreteArray(14.0)>
|<- sq_add_numpy = Traced<ConcreteArray(14.0)>
grad(sq_add_numpy) =  4.0


Notice that in the process of computing `grad`, JAX invokes `sq_add_numpy` and
`ma_numpy` with special arguments `ConcreteArray(...)` (described further 
below in this colab). 
It is important to remember that a JAX-traceable function must be able to 
operate not only on concrete arguments but also on special abstract arguments
that JAX may use to abstract the function execution.

The JAX traceability property is satisfied as long as the function is written 
in terms of JAX primitives. 

# Defining new JAX primitives

The right way to add support for multiple-add is in terms of existing
JAX primitives, as shown above. However, in order to demonstrate how JAX
primitives work let us pretend that we want to add a new primitive to 
JAX for the multiply-add functionality.

In [0]:
from jax import core
ma_p = core.Primitive("ma")  # Create the primitive

@trace("ma_prim")
def ma_prim(x, y, z):
  """The JAX-traceable way to use the JAX primitive."""
  return ma_p.bind(x, y, z)

@trace("sq_add_prim")
def sq_add_prim(a, b):
  """A square-add function implemented using the new JAX-primitive."""
  return ma_prim(a, a, b)

If we try to call the newly defined functions we get an error, because
we have not yet told JAX anything about the semantics of the new primitive.

In [0]:
with expectNotImplementedError():
  sq_add_prim(2., 10.)


call sq_add_prim(2.0, 10.0)
  call ma_prim(2.0, 2.0, 10.0)

Found expected exception:


Traceback (most recent call last):
  File "<ipython-input-103-409d9f078921>", line 2, in <module>
    sq_add_prim(2., 10.)
  File "<ipython-input-100-0ffadd93fbdc>", line 47, in func_wrapper
    res = func(*args)
  File "<ipython-input-102-d330544c991f>", line 12, in sq_add_prim
    return ma_prim(a, a, b)
NotImplementedError: Evaluation rule for 'ma' not implemented


## Primal evaluation rules




In [0]:
@trace("ma_impl")
def ma_impl(x, y, z):
  """Concrete implementation of the primitive.

  This function does not need to be JAX traceable.
  Args:
    x, y, z: the concrete arguments of the primitive. Will only be caled with 
      concrete values.
  Returns:
    the concrete result of the primitive.
  """
  # Note that we can use the original numpy, which is not JAX traceable
  return onp.add(onp.multiply(x, y), z)

# Now we register the primal implementation with JAX
ma_p.def_impl(ma_impl)

<function __main__.ma_impl>

In [0]:
assert sq_add_prim(2., 10.) == 14.

call sq_add_prim(2.0, 10.0)
  call ma_prim(2.0, 2.0, 10.0)
    call ma_impl(2.0, 2.0, 10.0)
    |<- ma_impl = 14.0
  |<- ma_prim = 14.0
|<- sq_add_prim = 14.0


## JIT

If we now try to use `jit` we get a `NotImplementedError`:

In [0]:
with expectNotImplementedError():
  api.jit(sq_add_prim)(2., 10.)

call sq_add_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)

Found expected exception:


Traceback (most recent call last):
  File "<ipython-input-106-11b289222564>", line 2, in <module>
    api.jit(sq_add_prim)(2., 10.)
  File "/usr/local/lib/python3.6/dist-packages/jax/api.py", line 147, in f_jitted
    out = xla.xla_call(flat_fun, *args_flat, device_assignment=device_assignment, backend=backend)
  File "/usr/local/lib/python3.6/dist-packages/jax/core.py", line 569, in call_bind
    outs = primitive.impl(f, *args, **params)
NotImplementedError: Abstract evaluation for 'ma' not implemented


### Abstract evaluation rules
In order to JIT the function, and for other transformations as well, 
JAX first evaluates it abstractly using only the 
shape and type of the arguments. This abstract evaluation serves multiple
purposes:
  * Gets the sequence of JAX primitives that are used in the computation. This 
  sequence will be compiled. 
  * Computes the shape and type of all vectors and operations used in the computation. 


For example, the abstraction of a vector with 3 elements may be `ShapedArray(float32[3])`, or `ConcreteArray([1., 2., 3.])`. 
In the latter case, JAX uses the actual concrete value wrapped as an abstract value. Nevertheless, the abstract evaluation is not supposed to look at the concrete values, but only at the shapes and types.


In [0]:
from jax import abstract_arrays
@trace("ma_abstract_eval")
def ma_abstract_eval(xs, ys, zs):
  """Abstract evaluation of the primitive.

  This function does not need to be JAX traceable. It will be invoked with
  abstractions of the actual arguments. 
  Args:
    xs, ys, zs: abstractions of the arguments.
  Result:
    a ShapedArray for the result of the primitive.
  """
  assert xs.shape == ys.shape
  assert xs.shape == zs.shape
  return abstract_arrays.ShapedArray(xs.shape, xs.dtype)

# Now we register the abstract evaluation with JAX
ma_p.def_abstract_eval(ma_abstract_eval)

<function __main__.ma_abstract_eval>

If we re-attempt to JIT, we see how the abstract evaluation proceeds, but
we get another error, about missing the actual XLA compilation rule:

In [0]:
with expectNotImplementedError():
  api.jit(sq_add_prim)(2., 10.)

call sq_add_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
    call ma_abstract_eval(ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]))
    |<- ma_abstract_eval = ShapedArray(float32[])
  |<- ma_prim = Traced<ShapedArray(float32[])>
|<- sq_add_prim = Traced<ShapedArray(float32[])>

Found expected exception:


Traceback (most recent call last):
  File "<ipython-input-108-11b289222564>", line 2, in <module>
    api.jit(sq_add_prim)(2., 10.)
  File "/usr/local/lib/python3.6/dist-packages/jax/api.py", line 147, in f_jitted
    out = xla.xla_call(flat_fun, *args_flat, device_assignment=device_assignment, backend=backend)
  File "/usr/local/lib/python3.6/dist-packages/jax/core.py", line 569, in call_bind
    outs = primitive.impl(f, *args, **params)
NotImplementedError: XLA translation rule for primitive 'ma' not found


### XLA Compilation rules

JAX compilation works by compiling each primitive into a graph of XLA operations.

This is biggest hurdle to adding new functionality to JAX, because the 
set of XLA operations is limited, and JAX already has pre-defined primitives
for each of them.

In [0]:
@trace("ma_xla_translation")
def ma_xla_translation(c, xc, yc, zc):
  """The compilation to XLA of the primitive.

  Given an XlaBuilder and XlaOps for each argument, return the XlaOp for the
  result of the function.

  Does not need to be a JAX-traceable function.
  """
  return c.Add(c.Mul(xc, yc), zc)

# Now we register the XLA compilation rule with JAX
# TODO: for GPU? and TPU?
from jax import xla
xla.backend_specific_translations['cpu'][ma_p] = ma_xla_translation

Now we succeed to JIT. Notice below that JAX first evaluates the function
abstractly, which triggers the `ma_abstract_eval` function, and 
then compiles the set of primitives it has encountered, including `ma`.
At this point JAX invokes `ma_xla_translation`.

In [0]:
assert api.jit(lambda x, y: sq_add_prim(x, y))(2., 10.) == 14.

call sq_add_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
    call ma_abstract_eval(ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]))
    |<- ma_abstract_eval = ShapedArray(float32[])
  |<- ma_prim = Traced<ShapedArray(float32[])>
|<- sq_add_prim = Traced<ShapedArray(float32[])>
call ma_xla_translation(<JaxComputationBuilder>, <XlaOp at 0x7fd1232a0df8>, <XlaOp at 0x7fd1232a0df8>, <XlaOp at 0x7fd1232a0e68>)
|<- ma_xla_translation = <XlaOp at 0x7fd1232a0068>


Below is another use of `jit` where we differentiate only
with respect to the first argument. Notice how the second argument to `sq_add_prim` is concrete, which leads
in the third argument to `ma_abstract_eval` being 
`ConcreteArray`. We see that `ma_abstract_eval` may be used with
both `ShapedArray` and `ConcreteArray`.

In [0]:
assert api.jit(lambda x, y: sq_add_prim(x, y), 
               static_argnums=1)(2., 10.) == 14.

call sq_add_prim(Traced<ShapedArray(float32[])>, 10.0)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, 10.0)
    call ma_abstract_eval(ShapedArray(float32[]), ShapedArray(float32[]), ConcreteArray(10.0))
    |<- ma_abstract_eval = ShapedArray(float32[])
  |<- ma_prim = Traced<ShapedArray(float32[])>
|<- sq_add_prim = Traced<ShapedArray(float32[])>
call ma_xla_translation(<JaxComputationBuilder>, <XlaOp at 0x7fd1233187d8>, <XlaOp at 0x7fd1233187d8>, <XlaOp at 0x7fd1232a2030>)
|<- ma_xla_translation = <XlaOp at 0x7fd1232a2068>


## Forward differentiation

JAX implements forward differentiation in the form of
a Jacobian-vector product (see the [JAX autodiff cookbook](https://colab.research.google.com/github/google/jax/blob/master/notebooks/autodiff_cookbook.ipynb#scrollTo=OMmi9cyhs1bj)).

If we attempt now to compute the `jvp` function we get an
error because we have not yet told JAX how to differentiate
the `ma` primitive.

In [0]:
# The second argument `(2., 10.)` are the argument values
# where we evaluate the Jacobian, and the third `(1., 1.)`
# are the values of the tangents for the arguments.
with expectNotImplementedError():
  api.jvp(sq_add_prim, (2., 10.), (1., 1.))


call sq_add_prim(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(10.0)>)
  call ma_prim(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(10.0)>)

Found expected exception:


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/jax/interpreters/ad.py", line 215, in process_primitive
    jvp = primitive_jvps[primitive]
KeyError: ma

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-112-09426a9a2915>", line 2, in <module>
    api.jvp(sq_add_prim, (2., 10.), (1., 1.))
  File "/usr/local/lib/python3.6/dist-packages/jax/api.py", line 909, in jvp
    out_primals, out_tangents = ad.jvp(flat_fun).call_wrapped(ps_flat, ts_flat)
  File "/usr/local/lib/python3.6/dist-packages/jax/linear_util.py", line 165, in call_wrapped
    ans = self.f(*args, **dict(self.params, **kwargs))
NotImplementedError: Forward-mode differentiation rule for 'ma' not implemented


In [0]:
from jax import ad


@trace("ma_value_and_jvp")
def ma_value_and_jvp(arg_values, arg_tangents):
  """Evaluates the primal output and the tangents (Jacobian-vector product).

  Given values of the arguments and perturbation of the arguments (tangents), 
  compute the output of the primitive and the perturbation of the output.

  This method must be JAX-traceable. JAX may invoke it with abstract values 
  for the arguments and tangents.

  Args:
    arg_values: a tuple of arguments
    arg_tangents: a tuple with the tangents of the arguments. The tuple has 
      the same length as the arg_values. Some of the tangents may also be the 
      special value ad.Zero to specify a zero tangent.
  Returns:
     a pair of the primal output and the tangent.
  """
  x, y, z = arg_values
  xt, yt, zt = arg_tangents
  _trace("Primal evaluation:")
  # Now we have a JAX-traceable computation of the output. 
  # Normally, we can use the ma primtive itself to compute the primal output. 
  primal_out = ma_prim(x, y, z)
  
  _trace("Tangent evaluation:")
  # We must use a JAX-traceable way to compute the tangent. It turns out that 
  # the output tangent can be computed as (xt * y + x * yt + zt),
  # which we can implement in a JAX-traceable way using the same "ma_prim" primitive.
  
  # We do need to deal specially with Zero. Here we just turn it into a 
  # proper tensor of 0s (of the same shape as 'x'). 
  # An alternative would be to check for Zero and perform algebraic 
  # simplification of the output tangent computation.
  def make_zero(tan):
    return lax.zeros_like_array(x) if tan is ad.zero else tan  
  
  output_tangent = ma_prim(make_zero(xt), y, ma_prim(x, make_zero(yt), make_zero(zt)))
  return (primal_out, output_tangent)

# Register the forward differentiation rule with JAX 
ad.primitive_jvps[ma_p] = ma_value_and_jvp

In [0]:
# Tangent is: xt*y + x*yt + zt = 1.*2. + 2.*1. + 1. = 5.
assert api.jvp(sq_add_prim, (2., 10.), (1., 1.)) == (14., 5.)

call sq_add_prim(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(10.0)>)
  call ma_prim(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(10.0)>)
    call ma_value_and_jvp((2.0, 2.0, 10.0), (1.0, 1.0, 1.0))
      Primal evaluation:
      call ma_prim(2.0, 2.0, 10.0)
        call ma_impl(2.0, 2.0, 10.0)
        |<- ma_impl = 14.0
      |<- ma_prim = 14.0
      Tangent evaluation:
      call ma_prim(2.0, 1.0, 1.0)
        call ma_impl(2.0, 1.0, 1.0)
        |<- ma_impl = 3.0
      |<- ma_prim = 3.0
      call ma_prim(1.0, 2.0, 3.0)
        call ma_impl(1.0, 2.0, 3.0)
        |<- ma_impl = 5.0
      |<- ma_prim = 5.0
    |<- ma_value_and_jvp = (14.0, 5.0)
  |<- ma_prim = Traced<ConcreteArray(14.0)>
|<- sq_add_prim = Traced<ConcreteArray(14.0)>


TO EXPLAIN: 
  * Why is JAX using ConcreteArray in sq_add_prim? There is no abstract evaluation going on here.
  * Not sure how to explain that ma_prim is invoked with ConcreteValue, yet
  we do not call the ma_abstract_eval.
  * I think it would be useful to show the JAXPR here
 

### JIT of forward differentiation

We can apply JIT to the forward differentiation function:

In [0]:
assert api.jit(lambda arg_values, arg_tangents: 
                   api.jvp(sq_add_prim, arg_values, arg_tangents))(
         (2., 10.), (1., 1.)) == (14., 5.)

call sq_add_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
    call ma_value_and_jvp((Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>), (Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>))
      Primal evaluation:
      call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
        call ma_abstract_eval(ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]))
        |<- ma_abstract_eval = ShapedArray(float32[])
      |<- ma_prim = Traced<ShapedArray(float32[])>
      Tangent evaluation:
      call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
        call ma_abstract_eval(ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[

Notice that first we evaluate `ma_value_and_jvp` abstractly, which in turn
evaluates abstractly both the primal and the tanger evaluation (a total of 
3 invocations of the `ma` primitive). Then we compile the 3 occurrences
of the primitive.

## Reverse differentiation

If we attempt now to use reverse differentiation we
see that JAX starts by using the `ma_value_and_jvp` to 
compute the forward differentiation for abstract values, but then runs
into a `NotImplementedError`. 

When computing the reverse differentiation JAX first does abstract evaluation
of the forward differentiation code `ma_value_and_jvp` to obtain a 
trace of primitives that compute the output tangent. 
Observe that JAX performs this abstract evaluation with concrete values
for the differentiation point, and abstract values for the tangents. 
Observe also that JAX uses the special abstract tangent value `Zero` for
the tangent corresponding to the 3rd argument of `ma`. This reflects the 
fact that we do not differentiate w.r.t. the 2nd argument to `sq_add_prim`,
which flow to 3rd argument to `ma_prim`.

Observe also that during the abstract evaluation of the tangent we pass the 
value 0.0 as the tangent for the 3rd argument. This is due to the use
of the `make_zero` function in the definition of `ma_value_and_jvp`.

In [0]:
# This is reverse differentiation w.r.t. the first argument of sq_add_prim
with expectNotImplementedError():
  api.grad(sq_add_prim)(2., 10.)

call sq_add_prim(Traced<ConcreteArray(2.0)>, 10.0)
  call ma_prim(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, 10.0)
    call ma_value_and_jvp((Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, 10.0), (Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Zero))
      Primal evaluation:
      call ma_prim(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, 10.0)
        call ma_impl(2.0, 2.0, 10.0)
        |<- ma_impl = 14.0
      |<- ma_prim = 14.0
      Tangent evaluation:
      call ma_prim(Traced<ConcreteArray(2.0)>, Traced<ShapedArray(float32[])>, 0.0)
        call ma_abstract_eval(ConcreteArray(2.0), ShapedArray(float32[]), ConcreteArray(0.0))
        |<- ma_abstract_eval = ShapedArray(float32[])
      |<- ma_prim = Traced<ShapedArray(float32[])>
      call ma_prim(Traced<ShapedArray(float32[])>, Traced<ConcreteArray(2.0)>, Traced<ShapedArray(float32[])>)
        call ma_abstract_eval(ShapedArray(float32[]), ConcreteArray(2.0), ShapedArray(float32[]

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/jax/interpreters/ad.py", line 196, in get_primitive_transpose
    return primitive_transposes[p]
KeyError: ma

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-116-98e2af8b2d4d>", line 2, in <module>
    api.grad(sq_add_prim)(2., 10.)
  File "/usr/local/lib/python3.6/dist-packages/jax/api.py", line 338, in grad_f
    _, g = value_and_grad_f(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jax/api.py", line 396, in value_and_grad_f
    g = vjp_py(onp.ones((), dtype=dtype))
NotImplementedError: Reverse-mode differentiation rule for 'ma' not implemented


The above error is because there is a missing piece for JAX to be able
to use the forward differentiation code to compute reverse differentiation. 

### Transposition


As explained above, when computing reverse differentiation JAX obtains
a trace of primitives that compute the tangent using forward differentiation.
Then, **JAX interprets this trace abstractly backwards** and for each 
primitive it applies a **transposition** rule.

To understand what is going on, consider for now a simpler example of the function "f(x, y) = x * y + y". Assume we need to differentiate at the point `(2., 4.)`. JAX will produce the following JVP tangent calculation of `ft` from the tangents of the input `xt` and `yt`:
```
   a = xt * 4.
   b = 2. * yt
   c = a + b
   ft = c + yt
```

By construction, the tangent calculation is always linear in the input tangents. 
When the tangent calculation uses non-linear operations, such as multiplication,
one or more of the operands is constant such that the computation is still linear in the non-constant arguments.

JAX will produce the reverse differentiation computation by processing the
JVP computation backwards. For each operation in the tangent computation,
it accumulates the cotangents
of the variables used by the operation, using the cotangent of the result
of the operation:
```
  # Initialize cotangents of inputs and intermediate vars
  xct = yct = act = bct = cct = 0
  # Initialize cotangent of the output
  fct = 1.
  # Process "ft = c + yt"
  cct += fct
  yct += fct
  # Process "c = a + b"
  act += cct
  bct += cct
  # Process "b = 2. * yt"
  yct += 2. * bct
  # Process "a = xt * 4."
  xct += act * 4.
```

One can verify that this computation produces `xct = 4.` and `yct = 3.`, which 
are the partial derivatives of the function f. 

JAX knows for each primitive that may appear in a JVP calculation how to transpose it. Conceptually, if the primitive `p(x, y, z)` is linear in the arguments `y` and `z` for a constant value of `x`, e.g., `p(x, y, z) = y*cy + z*cz`, then the transposition of the primitive is:
```
p_transpose(out_ct, x, _, _) = (None, out_ct*cy, out_ct*cz)
```

Notice that `p_transpose` takes the cotangent of the output of the primitive and a value corresponding to each argument of the primitive. For the linear arguments, the transposition gets an undefined `_` value, and for the constant
arguments it gets the actual constants. The transposition returns a cotangent value for each argument of the primitive, with the value `None` returned 
for the constant arguments.

In particular, 
```
 add_transpose(out_ct, _, _) = (out_ct, out_ct)
 mult_transpose(out_ct, x, _) = (None, x * out_ct)
 mult_transpose(out_ct, _, y) = (out_ct * y, None)
```





In [0]:
@trace("ma_transpose")
def ma_transpose(ct, x, y, z):
  """Evaluates the transpose of a linear primitive.

  This method is only used when computing the backward gradient following 
  value_and_jvp, and is only needed for primitives that are used in the JVP 
  calculation for some other primitive. We need transposition for ma_prim, 
  because we have used ma_prim in the computation of the output_tangent in 
  ma_value_and_jvp.

  In our case, ma is not a linear primitive. However, it is used linearly 
  w.r.t. tangents in ma_value_and_jvp:
       output_tangent(xt, yt, zt) = ma_prim(xt, y, ma_prim(x, yt, zt)
  
  Always one of the first two multiplicative arguments are constants.

  Args:
      ct: the cotangent of the output of the primitive.
      x, y, z: values of the arguments. The arguments that are used linearly
        get the value ad_undefined_primal. The other arguments get a constant
        value.
  Returns:
      a tuple with the cotangent of the inputs, with the value None
      corresponding to the constant arguments.
  """
  if x is not ad.undefined_primal:
    # This use of ma is with a constant "x"
    assert y is ad.undefined_primal
    ct_y = ad.zero if ct is ad.zero else ma_prim(x, ct, lax.zeros_like_array(x))
    res = None, ct_y, ct
  else:
    # This use of ma is with a constant "y"
    assert x is ad.undefined_primal
    ct_x = ad.zero if ct is ad.zero else ma_prim(ct, y, lax.zeros_like_array(y))
    res = ct_x, None, ct
  return res


ad.primitive_transposes[ma_p] = ma_transpose

Now we can complete the run of the `grad`:

In [0]:
assert api.grad(sq_add_prim)(2., 10.) == 4.

call sq_add_prim(Traced<ConcreteArray(2.0)>, 10.0)
  call ma_prim(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, 10.0)
    call ma_value_and_jvp((Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, 10.0), (Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Zero))
      Primal evaluation:
      call ma_prim(Traced<ConcreteArray(2.0)>, Traced<ConcreteArray(2.0)>, 10.0)
        call ma_impl(2.0, 2.0, 10.0)
        |<- ma_impl = 14.0
      |<- ma_prim = 14.0
      Tangent evaluation:
      call ma_prim(Traced<ConcreteArray(2.0)>, Traced<ShapedArray(float32[])>, 0.0)
        call ma_abstract_eval(ConcreteArray(2.0), ShapedArray(float32[]), ConcreteArray(0.0))
        |<- ma_abstract_eval = ShapedArray(float32[])
      |<- ma_prim = Traced<ShapedArray(float32[])>
      call ma_prim(Traced<ShapedArray(float32[])>, Traced<ConcreteArray(2.0)>, Traced<ShapedArray(float32[])>)
        call ma_abstract_eval(ShapedArray(float32[]), ConcreteArray(2.0), ShapedArray(float32[]

Notice the two calls to `ma_transpose`. They correspond to the two
uses of `ma_prim` in the computation of the `output_tangent` in `ma_value_and_jvp`. The first call to transpose corresponds to the 
last use of `ma_prim`: `ma_prim(xt, y, ...)` where `y` is the constant 2.0.

### JIT of reverse differentiation 

Notice that the abstract evaluation of the `ma_value_and_jvp` is using only
abstract values, while in the absensce of JIT we used `ConcreteArray`.

In [0]:
assert api.jit(api.grad(sq_add_prim))(2., 10.) == 4.

call sq_add_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
    call ma_value_and_jvp((Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>), (Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Zero))
      Primal evaluation:
      call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
        call ma_abstract_eval(ShapedArray(float32[]), ShapedArray(float32[]), ShapedArray(float32[]))
        |<- ma_abstract_eval = ShapedArray(float32[])
      |<- ma_prim = Traced<ShapedArray(float32[])>
      Tangent evaluation:
      call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ConcreteArray(0.0)>)
        call ma_abstract_eval(ShapedArray(float32[]), ShapedArray(float32[]), ConcreteArray(0.0))
        |<- ma_abstract_eval =

## Batching

The batching transformation takes a point-wise computation and turns it
into a computation on vectors. If we try it right now, we get a `NotImplementedError`:

In [0]:
# The arguments are two vectors instead of two scalars
with expectNotImplementedError():
  api.vmap(sq_add_prim, in_axes=0, out_axes=0)(onp.array([2., 3.]),
                                               onp.array([10., 20.]))

call sq_add_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)

Found expected exception:


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/jax/interpreters/batching.py", line 162, in get_primitive_batcher
    return primitive_batchers[p]
KeyError: ma

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-120-e1a8bde26b60>", line 3, in <module>
    onp.array([10., 20.]))
  File "/usr/local/lib/python3.6/dist-packages/jax/api.py", line 609, in batched_fun
    lambda: _flatten_axes(out_tree(), out_axes))
  File "/usr/local/lib/python3.6/dist-packages/jax/interpreters/batching.py", line 41, in batch
    out_vals, out_dims = batch_fun(fun, in_vals, in_dims)
NotImplementedError: Batching rule for 'ma' not implemented


We need to tell JAX how to evaluate the batched version of the primitive. In this particular case, the `ma_prim` already operates pointwise for any dimension of input vectors. So the

In [0]:
from jax import batching


@trace("ma_batch")
def ma_batch(vector_arg_values, batch_axes):
  """Computes the batched version of the primitive.
  
  This must be a JAX-traceable function.
  
  Since the ma primitive already operates on tensors, to batch it we use
  the primitive itself. This works as long as the input are batched along
  the same axes. The result is batched along the axis that the inputs
  are batched.
  
  Args:
    vector_arg_values: a tuple of two arguments, each being a tensor of matching
      shape.
    batch_axes: the axes that are being batched. See vmap documentation.
  Returns:
    a tuple of the result, and the result axis that was batched. 
  """
  assert batch_axes[0] == batch_axes[1]
  assert batch_axes[0] == batch_axes[2]
  _trace("Using ma to compute the batch:")
  res = ma_prim(*vector_arg_values)
  return res, batch_axes[0]


batching.primitive_batchers[ma_p] = ma_batch

In [0]:
assert onp.allclose(api.vmap(sq_add_prim, in_axes=0, out_axes=0)(
  onp.array([2., 3.]),
  onp.array([10., 20.])),
  [14., 29.])

call sq_add_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
    call ma_batch(([2. 3.], [2. 3.], [10. 20.]), (0, 0, 0))
      Using ma to compute the batch:
      call ma_prim([2. 3.], [2. 3.], [10. 20.])
        call ma_impl([2. 3.], [2. 3.], [10. 20.])
        |<- ma_impl = [14. 29.]
      |<- ma_prim = [14. 29.]
    |<- ma_batch = ([14. 29.], 0)
  |<- ma_prim = Traced<ShapedArray(float32[])>
|<- sq_add_prim = Traced<ShapedArray(float32[])>


### JIT of batching

In [0]:
assert onp.allclose(api.jit(api.vmap(sq_add_prim, in_axes=0, out_axes=0))
                    (onp.array([2., 3.]),
                     onp.array([10., 20.])),
                    [14., 29.])

call sq_add_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
  call ma_prim(Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>, Traced<ShapedArray(float32[])>)
    call ma_batch((Traced<ShapedArray(float32[2])>, Traced<ShapedArray(float32[2])>, Traced<ShapedArray(float32[2])>), (0, 0, 0))
      Using ma to compute the batch:
      call ma_prim(Traced<ShapedArray(float32[2])>, Traced<ShapedArray(float32[2])>, Traced<ShapedArray(float32[2])>)
        call ma_abstract_eval(ShapedArray(float32[2]), ShapedArray(float32[2]), ShapedArray(float32[2]))
        |<- ma_abstract_eval = ShapedArray(float32[2])
      |<- ma_prim = Traced<ShapedArray(float32[2])>
    |<- ma_batch = (Traced<ShapedArray(float32[2])>, 0)
  |<- ma_prim = Traced<ShapedArray(float32[])>
|<- sq_add_prim = Traced<ShapedArray(float32[])>
call ma_xla_translation(<JaxComputationBuilder>, <XlaOp at 0x7fd1232f7a78>, <XlaOp at 0x7fd1232f7a78>, <XlaOp at 0x7fd1232f7260>)
|<- ma_xla_translation = <Xla